In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def conv2d(inputs, filters, kernel_size, strides=(1, 1), padding='valid', activation=None):
    return tf.layers.conv2d(
      inputs=inputs,
      filters=filters,
      kernel_size=kernel_size,
      padding=padding,
      activation=activation)
    
def max_pool(inputs, pool_size=[2, 2], strides=2):
    return tf.layers.max_pooling2d(inputs=inputs, pool_size=pool_size, strides=strides)
    
def dense(inputs, units, activation=None):
    return tf.layers.dense(inputs=inputs, units=units, activation=activation)

In [ ]:
def cnn_model_fn(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    conv1 = conv2d(input_layer, 32, [5, 5], padding='same', activation=tf.nn.relu)
    pool1 = max_pool(conv1)
    conv2 = conv2d(pool1, 64, [5, 5], padding='same', activation=tf.nn.relu)
    pool2 = max_pool(conv2)
    
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense1 = dense(pool2_flat, 1024, tf.nn.relu)
    dropout1 = tf.layers.dropout(dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    logits = dense(dropout1, 10)
    
    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
mnist = input_data.read_data_sets('MNIST_data')
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

In [ ]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="mnist_convnet_model")

In [ ]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=256,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=2000)

In [ ]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

In [ ]:
# Predict example
digit = eval_data[np.random.randint(10000)]
img = digit.reshape(28, 28)
plt.imshow(img, cmap='gray')
plt.show()

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": digit.reshape(1, -1)},
      num_epochs=1,
      shuffle=False)

predictions = list(mnist_classifier.predict(input_fn=predict_input_fn))[0]['probabilities']
plt.bar(range(0, 10), predictions)
plt.show()